In [1]:
## Created by Wentinn Liao

# Kalman Filter Research

In [2]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#@title Symlink Setup
import os

def ptpp(PATH: str) -> str: # Converts path to python path
    return PATH.replace('\\', '')

DRIVE_PATH = '/content/gdrive/My\ Drive/KF_RNN'
if not os.path.exists(ptpp(DRIVE_PATH)):
    %mkdir $DRIVE_PATH
SYM_PATH = '/content/KF_RNN'
if not os.path.exists(ptpp(SYM_PATH)):
    !ln -s $DRIVE_PATH $SYM_PATH
%cd $SYM_PATH

/content/gdrive/My Drive/KF_RNN


In [4]:
!pip install numpy imageio matplotlib scikit-learn torch==2.0.0 tensordict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.5/986.5 kB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 120.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [5]:
#@title Configure Jupyter Notebook
import matplotlib
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [6]:
#@title Library Setup
import numpy as np
import matplotlib.pyplot as plt
from typing import *
from argparse import Namespace
import copy
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as Fn
import torch.utils as ptu
import tensordict
from tensordict import TensorDict

from model.linear_system import LinearSystem
from model.kf import KF
from model.rnn_kf import RnnKF

from infrastructure import utils
from infrastructure.train import *

# seed = 7
# torch.manual_seed(seed)
# random.seed(seed)
torch.set_default_dtype(torch.double)

dev_type = 'cuda'
if dev_type == 'xla':
    !pip install torch-xla cloud-tpu-client https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl
    import torch_xla
    import torch_xla.core.xla_model as xm

plt.rcParams['figure.figsize'] = (7.0, 5.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Sample Complexity

In [7]:
#@title Model Parameters
ModelArgs = Namespace(
    S_D = 6,
    I_D = 6,
    O_D = 4,
    SNR = 2.
)

In [8]:
#@title Training Parameters

total_trace_lengths = sorted(set(torch.ceil(torch.pow(2, torch.arange(7., 12.5, 0.5))).to(int).tolist()))
num_traces = sorted(set(torch.ceil(torch.pow(2, torch.arange(0., 6.5, 0.5))).to(int).tolist()))

BaseTrainArgs = Namespace(
    # Dataset
    train_dataset_size = num_traces,
    valid_dataset_size = 100,
    total_train_sequence_length = total_trace_lengths,
    total_valid_sequence_length = 20000,

    # Batch sampling
    subsequence_length = 10,
    subsequence_initial_mode = "random",    # {"random", "replay_buffer"}
    sample_efficiency = 5,
    replay_buffer = 10,
    batch_size = 128,

    # Optimizer
    beta = 0.1,
    lr = 3e-4,
    momentum = 0.9,
    lr_decay = 0.99,
    optim_type = "Adam",                    # {"GD", "SGD", "SGDMomentum", "Adam"}
    l2_reg = 0.1,

    # Iteration
    iterations_per_epoch = 100,
    epochs = 20
)

In [9]:
#@title Experiment Parameters
BaseExperimentArgs = Namespace(
    n_systems = 16,
    ensemble_size = 1,
    log_frequency = 5,
    print_frequency = 20
)

In [ ]:
#@title Experiment Configurations
base_exp_name = 'SC'
output_dir = 'sample_complexity'
output_fname = 'result'

optim_configs = [
    ('SGDMomentum', {
        'lr': 2e-4
    }),
    ('Adam', {
        'lr': 2e-2
    }),
    ('GD', {
        'lr': 2e-4
    }),
]
system_configs = [
    ('', dict())
]

result = {}
for (optim_config_name, optim_config), (system_config_name, system_config) in itertools.product(
    optim_configs,
    system_configs
):
    TrainArgs = copy.copy(BaseTrainArgs)
    TrainArgs.__dict__.update(optim_config)
    TrainArgs.optim_type = optim_config_name

    ExperimentArgs = copy.copy(BaseExperimentArgs)
    ExperimentArgs.exp_name = f'Full{system_config_name}{optim_config_name}{base_exp_name}'
    ExperimentArgs.output_dir = output_dir

    Args = Namespace(
        model = ModelArgs,
        train = TrainArgs,
        experiment = ExperimentArgs
    )

    result[optim_config_name, system_config_name] = run_experiments(
        Args,
        ['total_train_sequence_length', 'train_dataset_size'],
        dev_type,
        system_kwargs=system_config,
        output_kwargs={'fname': output_fname}
    )[0]

FullSGDMomentumSC
Hyperparameters: {
    "model": {
        "S_D": 6,
        "I_D": 6,
        "O_D": 4,
        "SNR": 2.0
    },
    "train": {
        "train_dataset_size": [
            1,
            2,
            3,
            4,
            6,
            8,
            12,
            16,
            23,
            32,
            46,
            64
        ],
        "valid_dataset_size": 100,
        "total_train_sequence_length": [
            128,
            182,
            256,
            363,
            512,
            725,
            1024,
            1449,
            2048,
            2897,
            4096
        ],
        "total_valid_sequence_length": 20000,
        "subsequence_length": 10,
        "subsequence_initial_mode": "random",
        "sample_efficiency": 5,
        "replay_buffer": 10,
        "batch_size": 128,
        "beta": 0.1,
        "lr": 0.0002,
        "momentum": 0.9,
        "lr_decay": 0.99,
        "optim_type": "SGDMomentum",
  

In [ ]:
H, W = len(optim_configs), len(system_configs)
plt.rcParams['figure.figsize'] = (12.0, 20.0)
fig, axs = plt.subplots(H, 2 * W, subplot_kw={'projection': '3d'})

tail = 10
c = 5.
for (h, (optim_config_name, _)), (w, (system_config_name, _)) in itertools.product(
    enumerate(optim_configs),
    enumerate(system_configs)
):
    exp_name = system_config_name + optim_config_name + base_exp_name
    r = result[optim_config_name, system_config_name]

    training_loss, overfit_loss, validation_loss, irreducible_loss = (r[k].detach().cpu() for k in (
        'training_loss',
        'overfit_loss',
        'validation_loss',
        'irreducible_loss'
    ))

    normalized_validation_loss = torch.mean(validation_loss[:, :, :, :, -tail:], dim=-1) / irreducible_loss
    mean_normalized_validation_loss = torch.nan_to_num(torch.mean(normalized_validation_loss, dim=(2, 3)), nan=c)
    # mean_normalized_validation_loss[mean_normalized_validation_loss > 10000.] = c

    x_mesh, y_mesh = torch.meshgrid(
        torch.DoubleTensor(total_trace_lengths),
        torch.DoubleTensor(num_traces)
    )


    axs[h, 2 * w].view_init(elev=15., azim=90.)
    axs[h, 2 * w].scatter(x_mesh, y_mesh, mean_normalized_validation_loss, s=20, c=mean_normalized_validation_loss, cmap=plt.cm.YlOrRd_r)

    axs[h, 2 * w].set_xlabel('total_trace_length')
    axs[h, 2 * w].set_xscale('log')
    axs[h, 2 * w].set_ylabel('num_traces')
    axs[h, 2 * w].set_yscale('log')
    axs[h, 2 * w].set_zlim(bottom=0)
    axs[h, 2 * w].set_zlabel('validation_loss')
    axs[h, 2 * w].set_title(exp_name)
plt.show()

In [ ]:
H, W = len(optim_configs), len(system_configs)
plt.rcParams['figure.figsize'] = (12.0, 20.0)
fig, axs = plt.subplots(H, 2 * W, subplot_kw={'projection': '3d'})

tail = 10
c = 5.
for (h, (optim_config_name, _)), (w, (system_config_name, _)) in itertools.product(
    enumerate(optim_configs),
    enumerate(system_configs)
):
    exp_name = system_config_name + optim_config_name + base_exp_name
    r = result[optim_config_name, system_config_name]

    training_loss, overfit_loss, validation_loss, irreducible_loss = (r[k].detach().cpu() for k in (
        'training_loss',
        'overfit_loss',
        'validation_loss',
        'irreducible_loss'
    ))

    print(validation_loss.shape, irreducible_loss.shape)

    normalized_validation_loss = torch.mean(validation_loss[:, :, :, :, -tail:], dim=-1) / irreducible_loss
    mean_normalized_validation_loss = torch.nan_to_num(torch.mean(normalized_validation_loss, dim=(2, 3)), nan=c)
    mean_normalized_validation_loss[mean_normalized_validation_loss > 10000.] = c

    lr_factors_mesh, momentums_mesh = torch.meshgrid(torch.log2(torch.DoubleTensor(total_trace_lengths)), torch.log2(torch.DoubleTensor(num_traces)))

    # View 1
    axs[h, 2 * w].view_init(elev=15., azim=105.)
    axs[h, 2 * w].plot_surface(lr_factors_mesh[:, 2:], momentums_mesh[:, 2:], mean_normalized_validation_loss[:, 2:], c=mean_normalized_validation_loss, s=10, cmap=plt.cm.YlOrRd_r)

    axs[h, 2 * w].set_xlabel('total_trace_length')
    axs[h, 2 * w].set_ylabel('num_traces')
    axs[h, 2 * w].set_zlim(bottom=0)
    axs[h, 2 * w].set_zlabel('validation_loss')
    axs[h, 2 * w].set_title(exp_name)

    # View 2
    axs[h, 2 * w + 1].view_init(elev=15., azim=90.)
    axs[h, 2 * w + 1].plot_surface(lr_factors_mesh[:, 2:], momentums_mesh[:, 2:], mean_normalized_validation_loss[:, 2:], c=mean_normalized_validation_loss, s=10, cmap=plt.cm.YlOrRd_r)

    axs[h, 2 * w + 1].set_xlabel('total_trace_length')
    axs[h, 2 * w + 1].set_ylabel('num_traces')
    axs[h, 2 * w + 1].set_zlim(bottom=0)
    axs[h, 2 * w + 1].set_zlabel('validation_loss')
    axs[h, 2 * w + 1].set_title(exp_name)
plt.show()

In [ ]:
H, W = len(optim_configs), len(system_configs)
plt.rcParams['figure.figsize'] = (12.0, 20.0)
fig, axs = plt.subplots(H, W)

for h, (optim_config_name, _) in enumerate(optim_configs):
    for w, (system_config_name, _) in enumerate(system_configs):
        exp_name = system_config_name + optim_config_name + base_exp_name

        r = result[optim_config_name][system_config_name]

        training_loss, overfit_loss, validation_loss, irreducible_loss = (r[k].detach().cpu() for k in (
            'training_loss',
            'overfit_loss',
            'validation_loss',
            'irreducible_loss'
        ))

        x = torch.arange(training_loss.shape[-1], dtype=float)
        irreducible_loss = irreducible_loss[:, :1]

        axs[h, w].plot(x, torch.ones_like(x), linestyle='--', linewidth=0.5, color='black', label='normalized irreducible_loss')
        for lname in ('training_loss', 'overfit_loss', 'validation_loss'):
            loss = torch.mean(eval(lname), dim=1)
            normalized_loss = loss / irreducible_loss

            mean_normalized_loss = torch.mean(normalized_loss, dim=0)
            min_normalized_loss = torch.min(normalized_loss, dim=0)
            max_normalized_loss = torch.max(normalized_loss, dim=0)

            axs[h, w].plot(x, min_normalized_loss.values, linewidth=0.5, label=f'mean normalized {lname}')
        #     plt.fill_between(x, min_normalized_loss, max_normalized_loss, alpha=0.2)

        axs[h, w].set_xlabel('Batch')
        axs[h, w].set_ylabel('Normalized Loss')
        axs[h, w].set_title(exp_name)
        # axs[h, w].legend()
plt.show()

In [ ]:
H, W = len(optim_configs), len(system_configs)
plt.rcParams['figure.figsize'] = (12.0, 20.0)
fig, axs = plt.subplots(H, W)

tail = 10
cutoff = 100
for h, (optim_config_name, _) in enumerate(optim_configs):
    for w, (system_config_name, _) in enumerate(system_configs):
        exp_name = system_config_name + optim_config_name + base_exp_name

        r = result[optim_config_name][system_config_name]

        training_loss, overfit_loss, validation_loss, irreducible_loss = (r[k].detach().cpu() for k in (
            'training_loss',
            'overfit_loss',
            'validation_loss',
            'irreducible_loss'
        ))

        x = torch.arange(cutoff, training_loss.shape[-1], dtype=float)

        axs[h, w].plot(x, torch.ones_like(x), linestyle='--', linewidth=1., color='black', label='normalized irreducible_loss')
        normalized_overfit_loss = overfit_loss / irreducible_loss[:, :, None]
        normalized_validation_loss = validation_loss / irreducible_loss[:, :, None]

        N = len(irreducible_loss)
        for n, (overfit_loss, validation_loss) in list(enumerate(zip(normalized_overfit_loss, normalized_validation_loss)))[::2]:
            mean_overfit_loss = torch.mean(overfit_loss, dim=0)[cutoff:]
            mean_validation_loss = torch.mean(validation_loss, dim=0)[cutoff:]

            c = color(n, scale=N)
            axs[h, w].plot(x, mean_overfit_loss, linewidth=1., color=c, label=f'Experiment {n}')
            axs[h, w].plot(x, mean_validation_loss, linewidth=1., color=c)
            axs[h, w].fill_between(x, mean_overfit_loss, mean_validation_loss, alpha=0.1, color=c)

        axs[h, w].set_xlabel('Batch')
        axs[h, w].set_ylabel('Normalized Loss')
        axs[h, w].set_title(f'{exp_name} - Normalized Overfit Loss')
        # axs[h, w].legend()
plt.show()

In [ ]:
H, W = len(optim_configs), len(system_configs)
plt.rcParams['figure.figsize'] = (12.0, 20.0)
fig, axs = plt.subplots(H, W)

tail = 10
for h, (optim_config_name, _) in enumerate(optim_configs):
    for w, (system_config_name, _) in enumerate(system_configs):
        exp_name = system_config_name + optim_config_name + base_exp_name

        r = result[optim_config_name][system_config_name]

        training_loss, overfit_loss, validation_loss, irreducible_loss = (r[k].detach().cpu() for k in (
            'training_loss',
            'overfit_loss',
            'validation_loss',
            'irreducible_loss'
        ))

        normalized_overfit_loss = overfit_loss / irreducible_loss[:, :, None]
        normalized_validation_loss = validation_loss / irreducible_loss[:, :, None]

        irreducible_, indices = torch.sort(irreducible_loss[:, 0])
        tail_ = torch.mean(normalized_overfit_loss[:, :, -tail:], dim=-1)
        mean_ = torch.mean(tail_, dim=1)[indices]
        std_ = torch.std(tail_, dim=1)[indices]
        min_ = tail_[torch.arange(len(mean_)), torch.argmin(tail_, dim=1)][indices]
        max_ = tail_[torch.arange(len(mean_)), torch.argmax(tail_, dim=1)][indices]


        axs_twinx = axs[h, w].twinx()
        axs[h, w].plot(irreducible_, torch.zeros_like(mean_), linewidth=1., linestyle='--', marker='.', color='black', label=f'Mean loss')
        axs_twinx.plot(irreducible_, mean_, linewidth=1., marker='.', color='blue', label='mean overfit loss')

        axs[h, w].plot(irreducible_, min_ - mean_, linewidth=0.5, marker='.', color='turquoise')
        # axs[h, w].plot(mean_, max_ - mean_, linewidth=0.5, marker='.', color='turquoise')

        axs[h, w].fill_between(irreducible_, min_ - mean_, max_ - mean_, color='aquamarine', alpha=0.2, label='min-max')
        axs[h, w].fill_between(irreducible_, -std_, std_, color='aquamarine', alpha=0.5, label='1 std')

        axs[h, w].set_xlabel('Normalized mean loss per system')
        axs[h, w].set_title(f'{exp_name}')
        # axs[h, w].legend()
plt.show()